## 准备数据

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)
train_data, test_data = mnist_dataset()
print(train_data[0].shape)

(60000, 28, 28)


In [7]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [8]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.compat.v1.get_variable(name="W1", shape=(784, 100), initializer=tf.random_normal_initializer(), trainable=True)
        self.W2 = tf.compat.v1.get_variable(name="W2", shape=(100, 10), initializer=tf.random_normal_initializer(), trainable=True)
        self.b1 = tf.compat.v1.get_variable(name="b1", shape=(100, ),trainable=True)
        self.b2 = tf.compat.v1.get_variable(name="b2", shape=(10, ), trainable=True)
#         self.W1 = tf.Variable(tf.random.truncated_normal([784, 100], stddev=0.1), trainable=True)
#         self.W2 = tf.Variable(tf.random.truncated_normal([100, 10], stddev=0.1), trainable=True)
#         self.b1 = tf.Variable(tf.constant(0.1, shape=[100]), trainable=True)
#         self.b2 = tf.Variable(tf.constant(0.1, shape=[10]), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_image = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x_image, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2 #此处不加softmax
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)
    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    
    # 此处改用优化器进行优化
    optimizer.apply_gradients(zip(grads, trainable_vars))

#     for g, v in zip(grads, trainable_vars):
#         v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3802025 ; accuracy 0.1143
epoch 1 : loss 2.3169358 ; accuracy 0.14356667
epoch 2 : loss 2.259407 ; accuracy 0.1941
epoch 3 : loss 2.2057807 ; accuracy 0.23691666
epoch 4 : loss 2.1543567 ; accuracy 0.285
epoch 5 : loss 2.103952 ; accuracy 0.34595
epoch 6 : loss 2.0538547 ; accuracy 0.40596667
epoch 7 : loss 2.003636 ; accuracy 0.45405
epoch 8 : loss 1.952961 ; accuracy 0.4926
epoch 9 : loss 1.9015715 ; accuracy 0.52556664
epoch 10 : loss 1.8492461 ; accuracy 0.5559667
epoch 11 : loss 1.7959135 ; accuracy 0.58891666
epoch 12 : loss 1.7416967 ; accuracy 0.62268335
epoch 13 : loss 1.686889 ; accuracy 0.65456665
epoch 14 : loss 1.6318784 ; accuracy 0.68513334
epoch 15 : loss 1.5770636 ; accuracy 0.70996666
epoch 16 : loss 1.5227565 ; accuracy 0.7306
epoch 17 : loss 1.4691617 ; accuracy 0.74575
epoch 18 : loss 1.416418 ; accuracy 0.75691664
epoch 19 : loss 1.364606 ; accuracy 0.76641667
epoch 20 : loss 1.3137909 ; accuracy 0.77346665
epoch 21 : loss 1.2640873 ; accuracy 0.7